# Fine-tuning a model on a token classification task

## Loading the dataset

## Preprocessing the data

In [1]:
from transformers import AutoTokenizer
import os
import optuna
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, AutoConfig
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np
from datasets import load_from_disk, concatenate_datasets
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Inicializando a variável particoes
particoes = {}
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
# atch_size = 8
print(particoes.keys())

label_all_tokens = True
# Carregar o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Caminho correto para as partições
base_path = "../models/"

# Carregar as partições diretamente do caminho especificado
for i in range(10):
    partition_path = f"{base_path}dataset_division_{i}"
    print(f"Verificando partição: {partition_path}")  # Para depuração
    
    if os.path.exists(partition_path):  # Verifica se o diretório existe
        try:
            particoes[f"particao_{i}"] = load_from_disk(partition_path)
        except Exception as e:
            print(f"Erro ao carregar a partição {partition_path}: {e}")
    else:
        print(f"Partição {partition_path} não encontrada!")  # Alerta se o diretório não existir  # Verificar quais partições foram carregadas corretamente
print("Partições carregadas:", list(particoes.keys()))


# Criar o mapeamento de label2id
label2id = {
    'O': 0,                     # Fora de entidade
    'B-ABSTRACCAO': 1,          # Início da entidade ABSTRACCAO
    'I-ABSTRACCAO': 2,          # Interior da entidade ABSTRACCAO
    'B-ACONTECIMENTO': 3,       # Início da entidade ACONTECIMENTO
    'I-ACONTECIMENTO': 4,       # Interior da entidade ACONTECIMENTO
    'B-COISA': 5,               # Início da entidade COISA
    'I-COISA': 6,               # Interior da entidade COISA
    'B-LOCAL': 7,               # Início da entidade LOCAL
    'I-LOCAL': 8,               # Interior da entidade LOCAL
    'B-OBRA': 9,                # Início da entidade OBRA
    'I-OBRA': 10,               # Interior da entidade OBRA
    'B-ORGANIZACAO': 11,        # Início da entidade ORGANIZACAO
    'I-ORGANIZACAO': 12,        # Interior da entidade ORGANIZACAO
    'B-PESSOA': 13,             # Início da entidade PESSOA
    'I-PESSOA': 14,             # Interior da entidade PESSOA
    'B-TEMPO': 15,              # Início da entidade TEMPO
    'I-TEMPO': 16,              # Interior da entidade TEMPO
    'B-VALOR': 17,              # Início da entidade VALOR
    'I-VALOR': 18,              # Interior da entidade VALOR
    'B-OUTRO' : 19,
    'I-OUTRO' : 20
}


# Exibir o mapeamento de labels
print(f"Label2ID mapping: {label2id}")

def tokenize_and_align_labels(examples):
    # Verificando se 'sentences' está presente nos dados
    if "sentences" not in examples:
        raise KeyError("A chave 'sentences' não foi encontrada nos dados.")

    tokenized_inputs = []
    labels = []

    for sentence in examples["sentences"]:
        # Tokenizando os tokens de cada sentença
        tokenized_sentence = tokenizer(sentence["tokens"], truncation=True, is_split_into_words=True, max_length=512)

        word_ids = tokenized_sentence.word_ids()  # Pega os ids dos tokens
        sentence_labels = sentence["labels"]  # Labels da sentença

        previous_word_idx = None
        label_ids = []          # Atribuindo as labels corretamente
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token (e.g., [CLS], [SEP])
            elif word_idx != previous_word_idx:
                # Verificando se a label existe em label2id
                label = sentence_labels[word_idx]
                if label not in label2id:
                    print(f"Label {label} não encontrada em label2id!")  # Print de depuração
                label_ids.append(label2id.get(label, -1))  # Usa -1 se a label não for encontrada
            else:
                label_ids.append(label2id.get(sentence_labels[word_idx], -1) if label_all_tokens else -100)

            previous_word_idx = word_idx

        tokenized_inputs.append(tokenized_sentence)
        labels.append(label_ids)

    # Atribuindo as labels ao tokenized_inputs
    for i, tokenized_sentence in enumerate(tokenized_inputs):
        tokenized_sentence["labels"] = labels[i]

    # Retorna os dados tokenizados e alinhados com as labels
    return {"input_ids": [x["input_ids"] for x in tokenized_inputs],
            "attention_mask": [x["attention_mask"] for x in tokenized_inputs],
            "labels": labels}

# Aplicar a tokenização a todas as partições de dados
for i in range(10):
    print(f"Tokenizando partição {i}...")
    particoes[f"particao_{i}"] = particoes[f"particao_{i}"].map(tokenize_and_align_labels, batched=True)

print("Tokenização aplicada com sucesso!")


dict_keys([])
Verificando partição: ../models/dataset_division_0
Verificando partição: ../models/dataset_division_1
Verificando partição: ../models/dataset_division_2
Verificando partição: ../models/dataset_division_3
Verificando partição: ../models/dataset_division_4
Verificando partição: ../models/dataset_division_5
Verificando partição: ../models/dataset_division_6
Verificando partição: ../models/dataset_division_7
Verificando partição: ../models/dataset_division_8
Verificando partição: ../models/dataset_division_9
Partições carregadas: ['particao_0', 'particao_1', 'particao_2', 'particao_3', 'particao_4', 'particao_5', 'particao_6', 'particao_7', 'particao_8', 'particao_9']
Label2ID mapping: {'O': 0, 'B-ABSTRACCAO': 1, 'I-ABSTRACCAO': 2, 'B-ACONTECIMENTO': 3, 'I-ACONTECIMENTO': 4, 'B-COISA': 5, 'I-COISA': 6, 'B-LOCAL': 7, 'I-LOCAL': 8, 'B-OBRA': 9, 'I-OBRA': 10, 'B-ORGANIZACAO': 11, 'I-ORGANIZACAO': 12, 'B-PESSOA': 13, 'I-PESSOA': 14, 'B-TEMPO': 15, 'I-TEMPO': 16, 'B-VALOR': 17, 'I

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Tokenizando partição 5...
Tokenizando partição 6...
Tokenizando partição 7...
Tokenizando partição 8...
Tokenizando partição 9...
Tokenização aplicada com sucesso!


## Fine-tuning the model

In [2]:
def criar_modelo(num_labels, attn_dropout, hidden_dropout):
    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels=num_labels,
        hidden_dropout_prob=hidden_dropout,
        attention_probs_dropout_prob=attn_dropout
    )
    return model

In [3]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

def objective(trial, train_set, val_set, num_labels):
    # Amostragem de hiperparâmetros
    lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    epochs = trial.suggest_categorical("epochs", [2, 3, 4, 8, 16, 32, 64, 128])
    attn_dropout = trial.suggest_uniform("attention_dropout", 0.0, 0.5)
    hidden_dropout = trial.suggest_uniform("hidden_dropout", 0.0, 0.5)
    weight_decay = trial.suggest_uniform("weight_decay", 0.01, 0.05)

    training_args = TrainingArguments(
        output_dir="./optuna_results",
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        logging_dir="./optuna_logs",
        logging_steps=10,
        report_to="none",
    )

    model = criar_modelo(num_labels, attn_dropout, hidden_dropout)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator= data_collator,
        train_dataset=train_set,
        eval_dataset=val_set,
        compute_metrics=compute_metrics
    )

    trainer.train()
    val_metrics = trainer.evaluate()

    return val_metrics["eval_f1_macro"]  # Otimização baseada no F1 macro


In [4]:
import os
import torch
import pandas as pd
import random
import numpy as np
from torch.utils.data import Subset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
import optuna
from optuna.samplers import TPESampler

# Garantir que os diretórios existam
checkpoints_dir = "./checkpoints"
logs_dir = "./logs_final"
final_results_dir = "./final_results"
optuna_db = "sqlite:///optuna_study.db"

os.makedirs(checkpoints_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)
os.makedirs(final_results_dir, exist_ok=True)

torch.cuda.empty_cache()

num_particoes = 10

# Função para obter uma amostra de 0.1% dos dados
def get_subset(dataset, fraction=1):
    total_size = len(dataset)
    subset_size = int(total_size * fraction)
    indices = random.sample(range(total_size), subset_size)
    return Subset(dataset, indices)

# Função para computar as métricas
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []

    # Considerando apenas as classes com prefixo 'B-'
    for pred, label in zip(predictions, labels):
        true_labels.extend([l for l in label if l != -100 and l % 2 == 1])  # Apenas 'B-...'
        pred_labels.extend([p for p, l in zip(pred, label) if l != -100 and l % 2 == 1])  # Apenas 'B-...'

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average=None, labels=[1, 3, 5, 7, 9, 11,13,15,17,19])  # 'B-...'
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(true_labels, pred_labels, average="macro")
    accuracy = accuracy_score(true_labels, pred_labels)

    return {
        "precision_per_class": precision.tolist(),
        "recall_per_class": recall.tolist(),
        "f1_per_class": f1.tolist(),
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "accuracy": accuracy
    }

# Carregar progresso se houver checkpoints
test_results = []
completed_iterations = set()

checkpoint_file = os.path.join(checkpoints_dir, "completed_iterations.txt")
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        completed_iterations = set(map(int, f.read().splitlines()))

num_labels = len(set(label for dataset in particoes.values() for sentence in dataset["sentences"] for label in sentence["labels"]))

import torch
import optuna
import os
import numpy as np
from transformers import TrainingArguments, Trainer
from optuna.samplers import TPESampler






In [5]:


# Definição da partição de validação
validacao_set = particoes["particao_7"]

# Definir conjunto de treino excluindo a partição de validação
train_sets_validacao = [particoes[f"particao_{j}"] for j in range(num_particoes) if j != 7]
train_set_validacao = torch.utils.data.ConcatDataset(train_sets_validacao)

# Otimização de hiperparâmetros com Optuna na partição 7
study = optuna.create_study(direction="maximize", sampler=TPESampler(), storage=optuna_db, load_if_exists=True)
study.optimize(lambda trial: objective(trial, train_set_validacao, validacao_set, num_labels), n_trials=25)

best_hyperparams = study.best_params
print(f"Melhores hiperparâmetros na partição de validação: {best_hyperparams}")




[I 2025-04-30 19:45:55,927] A new study created in RDB with name: no-name-0e0d2fb0-8bc8-46d3-ae87-2aa2b791266e
/tmp/ipykernel_554315/5382319.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_554315/5382319.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  attn_dropout = trial.suggest_uniform("attention_dropout", 0.0, 0.5)
/tmp/ipykernel_554315/5382319.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  hidden_dropout = trial.suggest_uniform("hidden_dropout", 0.0, 0.5)


Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,1.398500,0.937393,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,0.000000
2,0.770100,0.907349,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,0.000000
3,0.640200,0.832370,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,0.000000
4,0.555900,0.767487,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,0.000000
5,0.513700,0.746646,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,0.000000
6,0.486700,0.695796,"[0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0410958904109589, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.010204,0.001714,0.002935,0.005226
7,0.447200,0.684175,"[0.0, 0.0, 0.0, 0.2647058823529412, 0.0, 0.0, 0.11764705882352941, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.072, 0.0, 0.0, 0.021052631578947368, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.11320754716981132, 0.0, 0.0, 0.03571428571428571, 0.0, 0.0, 0.0]",0.027311,0.006647,0.010637,0.019164
8,0.444500,0.657844,"[0.0, 0.0, 0.0, 0.484375, 0.0, 0.4375, 0.23076923076923078, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.248, 0.0, 0.059322033898305086, 0.031578947368421054, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.328042328042328, 0.0, 0.1044776119402985, 0.05555555555555555, 0.0, 0.0, 0.0]",0.082332,0.024207,0.034863,0.071429
9,0.448600,0.640804,"[0.0, 0.0, 0.0, 0.582089552238806, 0.0, 0.45454545454545453, 0.4, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.312, 0.0, 0.0847457627118644, 0.08421052631578947, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.40625, 0.0, 0.14285714285714285, 0.1391304347826087, 0.0, 0.0, 0.0]",0.102617,0.034354,0.049160,0.099303
10,0.411700,0.624963,"[0.0, 0.0, 0.0, 0.6952380952380952, 0.0, 0.6046511627906976, 0.625, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.584, 0.0, 0.22033898305084745, 0.15789473684210525, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.6347826086956522, 0.0, 0.32298136645962733, 0.25210084033613445, 0.0, 0.0, 0.0]",0.137492,0.068731,0.086419,0.198606


/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-04-3

Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,No log,1.142012,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,0.000000
2,1.559400,0.874407,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,0.000000


...

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-04-3

Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.763500,0.561380,"[0.0, 0.0, 0.0, 0.3193916349809886, 0.0, 0.5, 0.42105263157894735, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.672, 0.0, 0.0847457627118644, 0.3368421052631579, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.4329896907216495, 0.0, 0.14492753623188406, 0.3742690058479532, 0.0, 0.0, 0.0]",0.103370,0.091132,0.079349,0.219512
2,0.370000,0.368229,"[0.0, 0.0, 0.0, 0.4711111111111111, 0.0, 0.6442307692307693, 0.875, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 0.848, 0.0, 0.5677966101694916, 0.6631578947368421, 0.0, 0.0375, 0.0]","[0.0, 0.0, 0.0, 0.6057142857142858, 0.0, 0.6036036036036037, 0.7544910179640718, 0.0, 0.07228915662650602, 0.0]",0.213596,0.151175,0.145436,0.416376
3,0.264900,0.253673,"[0.5714285714285714, 0.0, 0.5, 0.7909090909090909, 0.5555555555555556, 0.7815126050420168, 0.948051948051948, 0.8913043478260869, 0.8837209302325582, 0.0]","[0.2, 0.0, 0.15789473684210525, 0.696, 0.16129032258064516, 0.788135593220339, 0.7684210526315789, 0.7884615384615384, 0.475, 0.0]","[0.2962962962962963, 0.0, 0.24, 0.7404255319148936, 0.25, 0.7848101265822784, 0.8488372093023255, 0.8367346938775511, 0.6178861788617886, 0.0]",0.311710,0.212379,0.242894,0.606272
4,0.139400,0.196298,"[0.5555555555555556, 0.0, 0.6, 0.8512396694214877, 0.75, 0.853448275862069, 0.9493670886075949, 1.0, 0.9285714285714286, 0.0]","[0.5, 0.0, 0.47368421052631576, 0.824, 0.3870967741935484, 0.8389830508474576, 0.7894736842105263, 0.8461538461538461, 0.65, 0.0]","[0.5263157894736842, 0.0, 0.5294117647058824, 0.8373983739837398, 0.5106382978723404, 0.8461538461538461, 0.8620689655172413, 0.9166666666666666, 0.7647058823529411, 0.0]",0.341483,0.279442,0.304914,0.721254
5,0.112600,0.201421,"[0.5666666666666667, 1.0, 0.5882352941176471, 0.8849557522123894, 0.875, 0.8015267175572519, 0.9473684210526315, 0.8888888888888888, 0.9841269841269841, 0.0]","[0.425, 0.16666666666666666, 0.5263157894736842, 0.8, 0.45161290322580644, 0.8898305084745762, 0.7578947368421053, 0.9230769230769231, 0.775, 0.0]","[0.4857142857142857, 0.2857142857142857, 0.5555555555555556, 0.8403361344537815, 0.5957446808510638, 0.8433734939759037, 0.8421052631578947, 0.9056603773584906, 0.8671328671328671, 0.0]",0.396672,0.300810,0.327439,0.749129
6,0.060400,0.183873,"[0.625, 1.0, 0.6666666666666666, 0.8879310344827587, 0.7272727272727273, 0.8524590163934426, 0.948051948051948, 0.9056603773584906, 0.96875, 0.0]","[0.5, 0.25, 0.631578947368421, 0.824, 0.7741935483870968, 0.8813559322033898, 0.7684210526315789, 0.9230769230769231, 0.775, 0.0]","[0.5555555555555556, 0.4, 0.6486486486486487, 0.8547717842323651, 0.75, 0.8666666666666667, 0.8488372093023255, 0.9142857142857143, 0.8611111111111112, 0.0]",0.421211,0.351535,0.372215,0.782230
7,0.054300,0.175627,"[0.6896551724137931, 1.0, 0.65, 0.8782608695652174, 0.8, 0.8429752066115702, 0.9487179487179487, 0.9074074074074074, 0.967741935483871, 0.0]","[0.5, 0.25, 0.6842105263157895, 0.808, 0.7741935483870968, 0.864406779661017, 0.7789473684210526, 0.9423076923076923, 0.75, 0.0]","[0.5797101449275363, 0.4, 0.6666666666666666, 0.8416666666666667, 0.7868852459016393, 0.8535564853556485, 0.8554913294797688, 0.9245283018867925, 0.8450704225352113, 0.0]",0.426931,0.352893,0.375199,0.777003
8,0.042300,0.177579,"[0.65625, 1.0, 0.5909090909090909, 0.8849557522123894, 0.75, 0.865546218487395, 0.9487179487179487, 0.9056603773584906, 0.96875, 0.0]","[0.525, 0.25, 0.6842105263157895, 0.8, 0.7741935483870968, 0.8728813559322034, 0.7789473684210526, 0.9230769230769231, 0.775, 0.0]","[0.5833333333333334, 0.4, 0.6341463414634146, 0.8403361344537815, 0.7619047619047619, 0.869198312236287, 0.8554913294797688, 0.9142857142857143, 0.8611111111111112, 0.0]",0.420599,0.354628,0.373323,0.780488


/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-04-3

Melhores hiperparâmetros na partição de validação: {'learning_rate': 8.697030188113791e-05, 'batch_size': 8, 'epochs': 32, 'attention_dropout': 0.13408745592200652, 'hidden_dropout': 0.006300243771831415, 'weight_decay': 0.038076052263962816}


In [5]:
test_results = []

# Hiperparâmetros otimizados na partição de validação
best_hyperparams = {
    'learning_rate': 8.697030188113791e-05,
    'batch_size': 8,
    'epochs': 32,
    'attention_dropout': 0.13408745592200652,
    'hidden_dropout': 0.006300243771831415,
    'weight_decay': 0.038076052263962816
}

for i in range(num_particoes):
    if i == 7:
        continue  # Ignorar a partição de validação
    
    test_set = particoes[f"particao_{i}"]
    train_sets = [particoes[f"particao_{j}"] for j in range(num_particoes) if j not in {i, 7}]
    
    # Concatenando os datasets de treino
    train_set = torch.utils.data.ConcatDataset(train_sets)
    train_set = get_subset(train_set, fraction=1)
    
    # Configuração de checkpoints
    checkpoint_path = os.path.join(checkpoints_dir, f"model_checkpoint_{i}")
    os.makedirs(checkpoint_path, exist_ok=True)  # Garantir que o diretório existe
    print(f"Os checkpoints serão salvos em: {checkpoint_path}")

    # Configuração do treinamento
    final_training_args = TrainingArguments(
        output_dir=checkpoint_path,
        evaluation_strategy="epoch",
        save_strategy="epoch",  # Salvar ao final de cada época
        save_total_limit=3,
        load_best_model_at_end=True,
        learning_rate=best_hyperparams["learning_rate"],
        per_device_train_batch_size=best_hyperparams["batch_size"],
        per_device_eval_batch_size=best_hyperparams["batch_size"],
        num_train_epochs=best_hyperparams["epochs"],  # Garantindo que seja maior que 1
        weight_decay=best_hyperparams["weight_decay"],
        logging_dir=logs_dir,
        logging_steps=10,
        report_to="none",
    ) 
    
    final_model = criar_modelo(num_labels, best_hyperparams["attention_dropout"], best_hyperparams["hidden_dropout"])
    
    final_trainer = Trainer(
        model=final_model,
        args=final_training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics
    )
    
    final_trainer.train()

    # Forçar o salvamento do modelo ao final do treinamento
    final_trainer.save_model(checkpoint_path)

    # Verificar se os arquivos foram salvos corretamente
    print(f"Arquivos salvos em {checkpoint_path}: {os.listdir(checkpoint_path)}")

    test_metrics = final_trainer.evaluate()
    test_results.append(test_metrics)

# Cálculo da média e variância das métricas finais
avg_test_results = {
    metric: np.mean([result[metric] for result in test_results]) for metric in test_results[0]
}
var_test_results = {
    metric: np.var([result[metric] for result in test_results]) for metric in test_results[0]
}

# Criar DataFrame para a tabela desejada
df = pd.DataFrame({
    "abstraccao_precision": [p["eval_precision_per_class"][0] for p in test_results],
    "abstraccao_recall": [r["eval_recall_per_class"][0] for r in test_results],
    "abstraccao_f1": [f["eval_f1_per_class"][0] for f in test_results],

    "acontecimento_precision": [p["eval_precision_per_class"][1] for p in test_results],
    "acontecimento_recall": [r["eval_recall_per_class"][1] for r in test_results],
    "acontecimento_f1": [f["eval_f1_per_class"][1] for f in test_results],

    "coisa_precision": [p["eval_precision_per_class"][2] for p in test_results],
    "coisa_recall": [r["eval_recall_per_class"][2] for r in test_results],
    "coisa_f1": [f["eval_f1_per_class"][2] for f in test_results],

    "local_precision": [p["eval_precision_per_class"][3] for p in test_results],
    "local_recall": [r["eval_recall_per_class"][3] for r in test_results],
    "local_f1": [f["eval_f1_per_class"][3] for f in test_results],

    "obra_precision": [p["eval_precision_per_class"][4] for p in test_results],
    "obra_recall": [r["eval_recall_per_class"][4] for r in test_results],
    "obra_f1": [f["eval_f1_per_class"][4] for f in test_results],

    "organizacao_precision": [p["eval_precision_per_class"][5] for p in test_results],
    "organizacao_recall": [r["eval_recall_per_class"][5] for r in test_results],
    "organizacao_f1": [f["eval_f1_per_class"][5] for f in test_results],

    "pessoa_precision": [p["eval_precision_per_class"][6] for p in test_results],
    "pessoa_recall": [r["eval_recall_per_class"][6] for r in test_results],
    "pessoa_f1": [f["eval_f1_per_class"][6] for f in test_results],

    "tempo_precision": [p["eval_precision_per_class"][7] for p in test_results],
    "tempo_recall": [r["eval_recall_per_class"][7] for r in test_results],
    "tempo_f1": [f["eval_f1_per_class"][7] for f in test_results],

    "valor_precision": [p["eval_precision_per_class"][8] for p in test_results],
    "valor_recall": [r["eval_recall_per_class"][8] for r in test_results],
    "valor_f1": [f["eval_f1_per_class"][8] for f in test_results],

    "outro_precision": [p["eval_precision_per_class"][9] for p in test_results],
    "outro_recall": [r["eval_recall_per_class"][9] for r in test_results],
    "outro_f1": [f["eval_f1_per_class"][9] for f in test_results],

    "precision_macro": [p["eval_precision_macro"] for p in test_results],
    "recall_macro": [r["eval_recall_macro"] for r in test_results],
    "f1_macro": [f["eval_f1_macro"] for f in test_results],
    "accuracy": [a["eval_accuracy"] for a in test_results],
})


print(df)
print("Média das métricas:", avg_test_results)
print("Variância das métricas:", var_test_results)


import numpy as np
import pandas as pd

df_media_variancia = pd.DataFrame({
    # Abstração
    "abstracao_precision_mean": [np.mean([result["eval_precision_per_class"][0] for result in test_results])],
    "abstracao_precision_var": [np.var([result["eval_precision_per_class"][0] for result in test_results])],
    "abstracao_recall_mean": [np.mean([result["eval_recall_per_class"][0] for result in test_results])],
    "abstracao_recall_var": [np.var([result["eval_recall_per_class"][0] for result in test_results])],
    "abstracao_f1_mean": [np.mean([result["eval_f1_per_class"][0] for result in test_results])],
    "abstracao_f1_var": [np.var([result["eval_f1_per_class"][0] for result in test_results])],

    # Acontecimento
    "acontecimento_precision_mean": [np.mean([result["eval_precision_per_class"][1] for result in test_results])],
    "acontecimento_precision_var": [np.var([result["eval_precision_per_class"][1] for result in test_results])],
    "acontecimento_recall_mean": [np.mean([result["eval_recall_per_class"][1] for result in test_results])],
    "acontecimento_recall_var": [np.var([result["eval_recall_per_class"][1] for result in test_results])],
    "acontecimento_f1_mean": [np.mean([result["eval_f1_per_class"][1] for result in test_results])],
    "acontecimento_f1_var": [np.var([result["eval_f1_per_class"][1] for result in test_results])],

    # Coisa
    "coisa_precision_mean": [np.mean([result["eval_precision_per_class"][2] for result in test_results])],
    "coisa_precision_var": [np.var([result["eval_precision_per_class"][2] for result in test_results])],
    "coisa_recall_mean": [np.mean([result["eval_recall_per_class"][2] for result in test_results])],
    "coisa_recall_var": [np.var([result["eval_recall_per_class"][2] for result in test_results])],
    "coisa_f1_mean": [np.mean([result["eval_f1_per_class"][2] for result in test_results])],
    "coisa_f1_var": [np.var([result["eval_f1_per_class"][2] for result in test_results])],

    # Local
    "local_precision_mean": [np.mean([result["eval_precision_per_class"][3] for result in test_results])],
    "local_precision_var": [np.var([result["eval_precision_per_class"][3] for result in test_results])],
    "local_recall_mean": [np.mean([result["eval_recall_per_class"][3] for result in test_results])],
    "local_recall_var": [np.var([result["eval_recall_per_class"][3] for result in test_results])],
    "local_f1_mean": [np.mean([result["eval_f1_per_class"][3] for result in test_results])],
    "local_f1_var": [np.var([result["eval_f1_per_class"][3] for result in test_results])],

    # Obra
    "obra_precision_mean": [np.mean([result["eval_precision_per_class"][4] for result in test_results])],
    "obra_precision_var": [np.var([result["eval_precision_per_class"][4] for result in test_results])],
    "obra_recall_mean": [np.mean([result["eval_recall_per_class"][4] for result in test_results])],
    "obra_recall_var": [np.var([result["eval_recall_per_class"][4] for result in test_results])],
    "obra_f1_mean": [np.mean([result["eval_f1_per_class"][4] for result in test_results])],
    "obra_f1_var": [np.var([result["eval_f1_per_class"][4] for result in test_results])],

    # Organização
    "organizacao_precision_mean": [np.mean([result["eval_precision_per_class"][5] for result in test_results])],
    "organizacao_precision_var": [np.var([result["eval_precision_per_class"][5] for result in test_results])],
    "organizacao_recall_mean": [np.mean([result["eval_recall_per_class"][5] for result in test_results])],
    "organizacao_recall_var": [np.var([result["eval_recall_per_class"][5] for result in test_results])],
    "organizacao_f1_mean": [np.mean([result["eval_f1_per_class"][5] for result in test_results])],
    "organizacao_f1_var": [np.var([result["eval_f1_per_class"][5] for result in test_results])],

    # Pessoa
    "pessoa_precision_mean": [np.mean([result["eval_precision_per_class"][6] for result in test_results])],
    "pessoa_precision_var": [np.var([result["eval_precision_per_class"][6] for result in test_results])],
    "pessoa_recall_mean": [np.mean([result["eval_recall_per_class"][6] for result in test_results])],
    "pessoa_recall_var": [np.var([result["eval_recall_per_class"][6] for result in test_results])],
    "pessoa_f1_mean": [np.mean([result["eval_f1_per_class"][6] for result in test_results])],
    "pessoa_f1_var": [np.var([result["eval_f1_per_class"][6] for result in test_results])],

    # Tempo
    "tempo_precision_mean": [np.mean([result["eval_precision_per_class"][7] for result in test_results])],
    "tempo_precision_var": [np.var([result["eval_precision_per_class"][7] for result in test_results])],
    "tempo_recall_mean": [np.mean([result["eval_recall_per_class"][7] for result in test_results])],
    "tempo_recall_var": [np.var([result["eval_recall_per_class"][7] for result in test_results])],
    "tempo_f1_mean": [np.mean([result["eval_f1_per_class"][7] for result in test_results])],
    "tempo_f1_var": [np.var([result["eval_f1_per_class"][7] for result in test_results])],

    # Valor
    "valor_precision_mean": [np.mean([result["eval_precision_per_class"][8] for result in test_results])],
    "valor_precision_var": [np.var([result["eval_precision_per_class"][8] for result in test_results])],
    "valor_recall_mean": [np.mean([result["eval_recall_per_class"][8] for result in test_results])],
    "valor_recall_var": [np.var([result["eval_recall_per_class"][8] for result in test_results])],
    "valor_f1_mean": [np.mean([result["eval_f1_per_class"][8] for result in test_results])],
    "valor_f1_var": [np.var([result["eval_f1_per_class"][8] for result in test_results])],

    # Outro
    "outro_precision_mean": [np.mean([result["eval_precision_per_class"][9] for result in test_results])],
    "outro_precision_var": [np.var([result["eval_precision_per_class"][9] for result in test_results])],
    "outro_recall_mean": [np.mean([result["eval_recall_per_class"][9] for result in test_results])],
    "outro_recall_var": [np.var([result["eval_recall_per_class"][9] for result in test_results])],
    "outro_f1_mean": [np.mean([result["eval_f1_per_class"][9] for result in test_results])],
    "outro_f1_var": [np.var([result["eval_f1_per_class"][9] for result in test_results])],


    # Cálculo dos valores Macro
    "precision_macro_mean": [np.mean([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(10)
    ])],  # Média de todas as precisões
    "precision_macro_var": [np.var([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(10)
    ])],

    "recall_macro_mean": [np.mean([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(10)
    ])],  # Média de todos os recalls
    "recall_macro_var": [np.var([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(10)
    ])],

    "f1_macro_mean": [np.mean([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(10)
    ])],  # Média de todos os F1
    "f1_macro_var": [np.var([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(10)
    ])],

    # Cálculo de Accuracy
    "accuracy_mean": [np.mean([result["eval_accuracy"] for result in test_results])],
    "accuracy_var": [np.var([result["eval_accuracy"] for result in test_results])]
})

# Exibindo a nova tabela com as médias e variâncias
pd.set_option("display.max_columns", None)  # Mostra todas as colunas
pd.set_option("display.width", None)        # Não quebra linha
print(df_media_variancia)

# Mapeamento de rótulos para índices (como você já tem)
label_map = {
    'O': 0,                     # Fora de entidade
    'B-ABSTRACCAO': 1,          # Início da entidade ABSTRACCAO
    'I-ABSTRACCAO': 2,          # Interior da entidade ABSTRACCAO
    'B-ACONTECIMENTO': 3,       # Início da entidade ACONTECIMENTO
    'I-ACONTECIMENTO': 4,       # Interior da entidade ACONTECIMENTO
    'B-COISA': 5,               # Início da entidade COISA
    'I-COISA': 6,               # Interior da entidade COISA
    'B-LOCAL': 7,               # Início da entidade LOCAL
    'I-LOCAL': 8,               # Interior da entidade LOCAL
    'B-OBRA': 9,                # Início da entidade OBRA
    'I-OBRA': 10,               # Interior da entidade OBRA
    'B-ORGANIZACAO': 11,        # Início da entidade ORGANIZACAO
    'I-ORGANIZACAO': 12,        # Interior da entidade ORGANIZACAO
    'B-PESSOA': 13,             # Início da entidade PESSOA
    'I-PESSOA': 14,             # Interior da entidade PESSOA
    'B-TEMPO': 15,              # Início da entidade TEMPO
    'I-TEMPO': 16,              # Interior da entidade TEMPO
    'B-VALOR': 17,              # Início da entidade VALOR
    'I-VALOR': 18,              # Interior da entidade VALOR
    'B-OUTRO' : 19,
    'I-OUTRO' : 20
}

# Gerar a tabela de erros
predictions = final_trainer.predict(test_set)
pred_labels = predictions.predictions.argmax(axis=-1)  # As predições com maior probabilidade
true_labels = predictions.label_ids

# Mapeamento inverso de índice para rótulo
index_to_label = {v: k for k, v in label_map.items()}

# Extrair as palavras e comparar com as predições
# Extrair as palavras e comparar com as predições
error_data = []
for idx, (true, pred) in enumerate(zip(true_labels, pred_labels)):
    for word_idx, (true_label, pred_label) in enumerate(zip(true, pred)):
        # Ignorar tokens com rótulo -100 (tokens de padding ou ignore)
        if true_label == -100:
            continue
        
        # Obter as labels correspondentes aos índices
        true_class = index_to_label.get(true_label, "Desconhecido")
        pred_class = index_to_label.get(pred_label, "Desconhecido")

        # Obter a palavra real a partir do ID de token
        word_id = test_set[idx]['input_ids'][word_idx]  # ID do token
        word = tokenizer.decode([word_id], skip_special_tokens=True)  # Decodifica para a palavra

        # Se a predição for errada, armazenar a palavra e as classificações
        if true_class != pred_class:
            error_data.append({
                'word': word,  # Agora 'word' é a palavra real
                'true_class': true_class,
                'pred_class': pred_class
            })

# Criar o DataFrame de erros
error_df = pd.DataFrame(error_data)

# Exibir o DataFrame com os erros cometidos
print(error_df)

# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/harem_mini_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")







Os checkpoints serão salvos em: ./checkpoints/model_checkpoint_0


/home/juliaribeiro/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.493000,0.406220,"[0.0, 0.0, 0.0, 0.8409090909090909, 0.0, 0.8490566037735849, 0.8979591836734694, 1.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 0.7816901408450704, 0.0, 0.6338028169014085, 0.3464566929133858, 0.4375, 0.06976744186046512, 0.0]","[0.0, 0.0, 0.0, 0.8102189781021898, 0.0, 0.7258064516129032, 0.5, 0.6086956521739131, 0.13043478260869565, 0.0]",0.286745,0.141826,0.173447,0.352201
2,0.245200,0.339656,"[0.0, 0.0, 0.0, 0.7784810126582279, 0.07526881720430108, 0.7283950617283951, 0.8376068376068376, 0.9142857142857143, 0.9696969696969697, 0.0]","[0.0, 0.0, 0.0, 0.8661971830985915, 0.3684210526315789, 0.8309859154929577, 0.7716535433070866, 0.6666666666666666, 0.7441860465116279, 0.0]","[0.0, 0.0, 0.0, 0.82, 0.125, 0.7763157894736842, 0.8032786885245902, 0.7710843373493976, 0.8421052631578947, 0.0]",0.268983,0.265507,0.258612,0.551887
3,0.150500,0.265143,"[0.0, 0.0, 0.7857142857142857, 0.8823529411764706, 0.11363636363636363, 0.7631578947368421, 0.9662921348314607, 0.8717948717948718, 0.9714285714285714, 0.0]","[0.0, 0.0, 0.15172413793103448, 0.8450704225352113, 0.2631578947368421, 0.8169014084507042, 0.6771653543307087, 0.7083333333333334, 0.7906976744186046, 0.0]","[0.0, 0.0, 0.2543352601156069, 0.8633093525179856, 0.15873015873015872, 0.7891156462585034, 0.7962962962962963, 0.7816091954022989, 0.8717948717948718, 0.0]",0.297465,0.236281,0.250844,0.564465
4,0.070500,0.231071,"[0.0, 0.0, 0.8108108108108109, 0.935251798561151, 0.2903225806451613, 0.8309859154929577, 1.0, 0.8863636363636364, 0.972972972972973, 0.0]","[0.0, 0.0, 0.6206896551724138, 0.9154929577464789, 0.47368421052631576, 0.8309859154929577, 0.8110236220472441, 0.8125, 0.8372093023255814, 0.0]","[0.0, 0.0, 0.703125, 0.9252669039145908, 0.36, 0.8309859154929577, 0.8956521739130435, 0.8478260869565217, 0.9, 0.0]",0.357919,0.331349,0.341429,0.732704
5,0.053000,0.229040,"[0.08, 0.0, 0.7196969696969697, 0.972972972972973, 0.3157894736842105, 0.7733333333333333, 0.8869565217391304, 0.8809523809523809, 0.9523809523809523, 0.0]","[0.07142857142857142, 0.0, 0.6551724137931034, 0.7605633802816901, 0.3157894736842105, 0.8169014084507042, 0.8031496062992126, 0.7708333333333334, 0.9302325581395349, 0.0]","[0.07547169811320754, 0.0, 0.6859205776173285, 0.8537549407114624, 0.3157894736842105, 0.7945205479452054, 0.8429752066115702, 0.8222222222222222, 0.9411764705882353, 0.0]",0.310116,0.284671,0.296213,0.704403
6,0.032100,0.235909,"[0.0, 0.0, 0.88, 0.8972602739726028, 0.3333333333333333, 0.7916666666666666, 0.896, 0.8837209302325582, 0.975609756097561, 0.0]","[0.0, 0.0, 0.6068965517241379, 0.9225352112676056, 0.3157894736842105, 0.8028169014084507, 0.8818897637795275, 0.7916666666666666, 0.9302325581395349, 0.0]","[0.0, 0.0, 0.7183673469387755, 0.9097222222222222, 0.32432432432432434, 0.7972027972027972, 0.8888888888888888, 0.8351648351648352, 0.9523809523809523, 0.0]",0.332799,0.308931,0.319179,0.742138
7,0.013900,0.239593,"[0.21428571428571427, 0.0, 0.8035714285714286, 0.9047619047619048, 0.5, 0.8028169014084507, 0.9532710280373832, 0.8837209302325582, 0.975, 0.0]","[0.10714285714285714, 0.0, 0.6206896551724138, 0.9366197183098591, 0.3157894736842105, 0.8028169014084507, 0.8031496062992126, 0.7916666666666666, 0.9069767441860465, 0.0]","[0.14285714285714285, 0.0, 0.7003891050583657, 0.9204152249134948, 0.3870967741935484, 0.8028169014084507, 0.8717948717948718, 0.8351648351648352, 0.9397590361445783, 0.0]",0.335413,0.293603,0.311127,0.735849
8,0.012500,0.254118,"[0.09090909090909091, 0.5, 0.8165137614678899, 0.927536231884058, 0.3333333333333333, 0.7073170731707317, 0.9532710280373832, 0.9024390243902439, 0.9523809523809523, 0.0]","[0.03571428571428571, 0.125, 0.6137931034482759, 0.9014084507042254, 0.42105263157894735, 0.8169014084507042, 0.8031496062992126, 0.7708333333333334, 0.9302325581395349, 0.0]","[0.05128205128205128, 0.2, 0.700787401574

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

Arquivos salvos em ./checkpoints/model_checkpoint_0: ['model.safetensors', 'checkpoint-120', 'checkpoint-39040', 'config.json', 'training_args.bin', 'checkpoint-38735']


...

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

Arquivos salvos em ./checkpoints/model_checkpoint_9: ['model.safetensors', 'checkpoint-30256', 'checkpoint-31232', 'config.json', 'training_args.bin', 'checkpoint-168']


   abstraccao_precision  abstraccao_recall  abstraccao_f1  \
0              0.080000           0.071429       0.075472   
1              0.918919           0.781609       0.844720   
2              0.800000           0.307692       0.444444   
3              0.366667           0.423077       0.392857   
4              0.781250           0.595238       0.675676   
5              0.700000           0.451613       0.549020   
6              1.000000           0.516129       0.680851   
7              0.911111           0.854167       0.881720   
8              0.727273           0.285714       0.410256   

   acontecimento_precision  acontecimento_recall  acontecimento_f1  \
0                 0.000000              0.000000          0.000000   
1                 0.500000              0.125000          0.200000   
2                 0.764706              1.000000          0.866667   
3                 0.000000              0.000000          0.000000   
4                 0.555556             

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

       word true_class     pred_class
0      High     B-OBRA  B-ORGANIZACAO
1      Time     I-OBRA  I-ORGANIZACAO
2     Oscar    B-OUTRO  B-ORGANIZACAO
3       Rec    B-LOCAL  B-ORGANIZACAO
4    ##ivil    B-LOCAL  B-ORGANIZACAO
..      ...        ...            ...
271      da   I-PESSOA              O
272   Filho   I-PESSOA   I-ABSTRACCAO
273    Cons          O        B-LOCAL
274   ##ola          O        B-LOCAL
275  ##dora          O        B-LOCAL

[276 rows x 3 columns]
Arquivo salvo em: /home/juliaribeiro/harem_mini_error_data.txt


In [6]:

import numpy as np
import pandas as pd

df_media_variancia = pd.DataFrame({
    # Abstração
    "abstracao_precision_mean": [np.mean([result["eval_precision_per_class"][0] for result in test_results])],
    "abstracao_precision_var": [np.var([result["eval_precision_per_class"][0] for result in test_results])],
    "abstracao_recall_mean": [np.mean([result["eval_recall_per_class"][0] for result in test_results])],
    "abstracao_recall_var": [np.var([result["eval_recall_per_class"][0] for result in test_results])],
    "abstracao_f1_mean": [np.mean([result["eval_f1_per_class"][0] for result in test_results])],
    "abstracao_f1_var": [np.var([result["eval_f1_per_class"][0] for result in test_results])],

    # Acontecimento
    "acontecimento_precision_mean": [np.mean([result["eval_precision_per_class"][1] for result in test_results])],
    "acontecimento_precision_var": [np.var([result["eval_precision_per_class"][1] for result in test_results])],
    "acontecimento_recall_mean": [np.mean([result["eval_recall_per_class"][1] for result in test_results])],
    "acontecimento_recall_var": [np.var([result["eval_recall_per_class"][1] for result in test_results])],
    "acontecimento_f1_mean": [np.mean([result["eval_f1_per_class"][1] for result in test_results])],
    "acontecimento_f1_var": [np.var([result["eval_f1_per_class"][1] for result in test_results])],

    # Coisa
    "coisa_precision_mean": [np.mean([result["eval_precision_per_class"][2] for result in test_results])],
    "coisa_precision_var": [np.var([result["eval_precision_per_class"][2] for result in test_results])],
    "coisa_recall_mean": [np.mean([result["eval_recall_per_class"][2] for result in test_results])],
    "coisa_recall_var": [np.var([result["eval_recall_per_class"][2] for result in test_results])],
    "coisa_f1_mean": [np.mean([result["eval_f1_per_class"][2] for result in test_results])],
    "coisa_f1_var": [np.var([result["eval_f1_per_class"][2] for result in test_results])],

    # Local
    "local_precision_mean": [np.mean([result["eval_precision_per_class"][3] for result in test_results])],
    "local_precision_var": [np.var([result["eval_precision_per_class"][3] for result in test_results])],
    "local_recall_mean": [np.mean([result["eval_recall_per_class"][3] for result in test_results])],
    "local_recall_var": [np.var([result["eval_recall_per_class"][3] for result in test_results])],
    "local_f1_mean": [np.mean([result["eval_f1_per_class"][3] for result in test_results])],
    "local_f1_var": [np.var([result["eval_f1_per_class"][3] for result in test_results])],

    # Obra
    "obra_precision_mean": [np.mean([result["eval_precision_per_class"][4] for result in test_results])],
    "obra_precision_var": [np.var([result["eval_precision_per_class"][4] for result in test_results])],
    "obra_recall_mean": [np.mean([result["eval_recall_per_class"][4] for result in test_results])],
    "obra_recall_var": [np.var([result["eval_recall_per_class"][4] for result in test_results])],
    "obra_f1_mean": [np.mean([result["eval_f1_per_class"][4] for result in test_results])],
    "obra_f1_var": [np.var([result["eval_f1_per_class"][4] for result in test_results])],

    # Organização
    "organizacao_precision_mean": [np.mean([result["eval_precision_per_class"][5] for result in test_results])],
    "organizacao_precision_var": [np.var([result["eval_precision_per_class"][5] for result in test_results])],
    "organizacao_recall_mean": [np.mean([result["eval_recall_per_class"][5] for result in test_results])],
    "organizacao_recall_var": [np.var([result["eval_recall_per_class"][5] for result in test_results])],
    "organizacao_f1_mean": [np.mean([result["eval_f1_per_class"][5] for result in test_results])],
    "organizacao_f1_var": [np.var([result["eval_f1_per_class"][5] for result in test_results])],

    # Pessoa
    "pessoa_precision_mean": [np.mean([result["eval_precision_per_class"][6] for result in test_results])],
    "pessoa_precision_var": [np.var([result["eval_precision_per_class"][6] for result in test_results])],
    "pessoa_recall_mean": [np.mean([result["eval_recall_per_class"][6] for result in test_results])],
    "pessoa_recall_var": [np.var([result["eval_recall_per_class"][6] for result in test_results])],
    "pessoa_f1_mean": [np.mean([result["eval_f1_per_class"][6] for result in test_results])],
    "pessoa_f1_var": [np.var([result["eval_f1_per_class"][6] for result in test_results])],

    # Tempo
    "tempo_precision_mean": [np.mean([result["eval_precision_per_class"][7] for result in test_results])],
    "tempo_precision_var": [np.var([result["eval_precision_per_class"][7] for result in test_results])],
    "tempo_recall_mean": [np.mean([result["eval_recall_per_class"][7] for result in test_results])],
    "tempo_recall_var": [np.var([result["eval_recall_per_class"][7] for result in test_results])],
    "tempo_f1_mean": [np.mean([result["eval_f1_per_class"][7] for result in test_results])],
    "tempo_f1_var": [np.var([result["eval_f1_per_class"][7] for result in test_results])],

    # Valor
    "valor_precision_mean": [np.mean([result["eval_precision_per_class"][8] for result in test_results])],
    "valor_precision_var": [np.var([result["eval_precision_per_class"][8] for result in test_results])],
    "valor_recall_mean": [np.mean([result["eval_recall_per_class"][8] for result in test_results])],
    "valor_recall_var": [np.var([result["eval_recall_per_class"][8] for result in test_results])],
    "valor_f1_mean": [np.mean([result["eval_f1_per_class"][8] for result in test_results])],
    "valor_f1_var": [np.var([result["eval_f1_per_class"][8] for result in test_results])],

    # Outro
    "outro_precision_mean": [np.mean([result["eval_precision_per_class"][9] for result in test_results])],
    "outro_precision_var": [np.var([result["eval_precision_per_class"][9] for result in test_results])],
    "outro_recall_mean": [np.mean([result["eval_recall_per_class"][9] for result in test_results])],
    "outro_recall_var": [np.var([result["eval_recall_per_class"][9] for result in test_results])],
    "outro_f1_mean": [np.mean([result["eval_f1_per_class"][9] for result in test_results])],
    "outro_f1_var": [np.var([result["eval_f1_per_class"][9] for result in test_results])],


    # Cálculo dos valores Macro
    "precision_macro_mean": [np.mean([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(10)
    ])],  # Média de todas as precisões
    "precision_macro_var": [np.var([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(10)
    ])],

    "recall_macro_mean": [np.mean([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(10)
    ])],  # Média de todos os recalls
    "recall_macro_var": [np.var([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(10)
    ])],

    "f1_macro_mean": [np.mean([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(10)
    ])],  # Média de todos os F1
    "f1_macro_var": [np.var([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(10)
    ])],

    # Cálculo de Accuracy
    "accuracy_mean": [np.mean([result["eval_accuracy"] for result in test_results])],
    "accuracy_var": [np.var([result["eval_accuracy"] for result in test_results])]
})

# Exibindo a nova tabela com as médias e variâncias
pd.set_option("display.max_columns", None)  # Mostra todas as colunas
pd.set_option("display.width", None)        # Não quebra linha
print(df_media_variancia)

# Mapeamento de rótulos para índices (como você já tem)
label_map = {
    'O': 0,                     # Fora de entidade
    'B-ABSTRACCAO': 1,          # Início da entidade ABSTRACCAO
    'I-ABSTRACCAO': 2,          # Interior da entidade ABSTRACCAO
    'B-ACONTECIMENTO': 3,       # Início da entidade ACONTECIMENTO
    'I-ACONTECIMENTO': 4,       # Interior da entidade ACONTECIMENTO
    'B-COISA': 5,               # Início da entidade COISA
    'I-COISA': 6,               # Interior da entidade COISA
    'B-LOCAL': 7,               # Início da entidade LOCAL
    'I-LOCAL': 8,               # Interior da entidade LOCAL
    'B-OBRA': 9,                # Início da entidade OBRA
    'I-OBRA': 10,               # Interior da entidade OBRA
    'B-ORGANIZACAO': 11,        # Início da entidade ORGANIZACAO
    'I-ORGANIZACAO': 12,        # Interior da entidade ORGANIZACAO
    'B-PESSOA': 13,             # Início da entidade PESSOA
    'I-PESSOA': 14,             # Interior da entidade PESSOA
    'B-TEMPO': 15,              # Início da entidade TEMPO
    'I-TEMPO': 16,              # Interior da entidade TEMPO
    'B-VALOR': 17,              # Início da entidade VALOR
    'I-VALOR': 18,              # Interior da entidade VALOR
    'B-OUTRO' : 19,
    'I-OUTRO' : 20
}

# Gerar a tabela de erros
predictions = final_trainer.predict(test_set)
pred_labels = predictions.predictions.argmax(axis=-1)  # As predições com maior probabilidade
true_labels = predictions.label_ids

# Mapeamento inverso de índice para rótulo
index_to_label = {v: k for k, v in label_map.items()}

# Extrair as palavras e comparar com as predições
# Extrair as palavras e comparar com as predições
error_data = []
for idx, (true, pred) in enumerate(zip(true_labels, pred_labels)):
    for word_idx, (true_label, pred_label) in enumerate(zip(true, pred)):
        # Ignorar tokens com rótulo -100 (tokens de padding ou ignore)
        if true_label == -100:
            continue
        
        # Obter as labels correspondentes aos índices
        true_class = index_to_label.get(true_label, "Desconhecido")
        pred_class = index_to_label.get(pred_label, "Desconhecido")

        # Obter a palavra real a partir do ID de token
        word_id = test_set[idx]['input_ids'][word_idx]  # ID do token
        word = tokenizer.decode([word_id], skip_special_tokens=True)  # Decodifica para a palavra

        # Se a predição for errada, armazenar a palavra e as classificações
        if true_class != pred_class:
            error_data.append({
                'word': word,  # Agora 'word' é a palavra real
                'true_class': true_class,
                'pred_class': pred_class
            })

# Criar o DataFrame de erros
error_df = pd.DataFrame(error_data)

# Exibir o DataFrame com os erros cometidos
print(error_df)

# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/harem_second_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")





   abstracao_precision_mean  abstracao_precision_var  abstracao_recall_mean  \
0                  0.831976                 0.016682               0.702957   

   abstracao_recall_var  abstracao_f1_mean  abstracao_f1_var  \
0              0.015191           0.758566           0.01329   

   acontecimento_precision_mean  acontecimento_precision_var  \
0                      0.788512                     0.012872   

   acontecimento_recall_mean  acontecimento_recall_var  acontecimento_f1_mean  \
0                    0.54827                  0.080185               0.580461   

   acontecimento_f1_var  coisa_precision_mean  coisa_precision_var  \
0              0.054639              0.866626             0.024291   

   coisa_recall_mean  coisa_recall_var  coisa_f1_mean  coisa_f1_var  \
0           0.633805          0.050853       0.707686      0.033252   

   local_precision_mean  local_precision_var  local_recall_mean  \
0              0.848587             0.009346           0.867532   

 

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


         word    true_class       pred_class
0     ##idade  B-ABSTRACCAO                O
1     ##idade  B-ABSTRACCAO                O
2    Programa        B-OBRA                O
3          de        I-OBRA                O
4         Tre        I-OBRA  I-ACONTECIMENTO
..        ...           ...              ...
322        Ma  B-ABSTRACCAO                O
323      ##il  B-ABSTRACCAO                O
324      Mark  I-ABSTRACCAO                O
325      ##et  I-ABSTRACCAO                O
326     ##ing  I-ABSTRACCAO                O

[327 rows x 3 columns]
Arquivo salvo em: /home/juliaribeiro/harem_first_error_data.txt


In [7]:
# Mapeamento de rótulos para índices (como você já tem)
label_map = {
    'O': 0,                     # Fora de entidade
    'B-ABSTRACCAO': 1,          # Início da entidade ABSTRACCAO
    'I-ABSTRACCAO': 2,          # Interior da entidade ABSTRACCAO
    'B-ACONTECIMENTO': 3,       # Início da entidade ACONTECIMENTO
    'I-ACONTECIMENTO': 4,       # Interior da entidade ACONTECIMENTO
    'B-COISA': 5,               # Início da entidade COISA
    'I-COISA': 6,               # Interior da entidade COISA
    'B-LOCAL': 7,               # Início da entidade LOCAL
    'I-LOCAL': 8,               # Interior da entidade LOCAL
    'B-OBRA': 9,                # Início da entidade OBRA
    'I-OBRA': 10,               # Interior da entidade OBRA
    'B-ORGANIZACAO': 11,        # Início da entidade ORGANIZACAO
    'I-ORGANIZACAO': 12,        # Interior da entidade ORGANIZACAO
    'B-PESSOA': 13,             # Início da entidade PESSOA
    'I-PESSOA': 14,             # Interior da entidade PESSOA
    'B-TEMPO': 15,              # Início da entidade TEMPO
    'I-TEMPO': 16,              # Interior da entidade TEMPO
    'B-VALOR': 17,              # Início da entidade VALOR
    'I-VALOR': 18,              # Interior da entidade VALOR
    'B-OUTRO' : 19,
    'I-OUTRO' : 20
}

# Gerar a tabela de erros
predictions = final_trainer.predict(test_set)
pred_labels = predictions.predictions.argmax(axis=-1)  # As predições com maior probabilidade
true_labels = predictions.label_ids

# Mapeamento inverso de índice para rótulo
index_to_label = {v: k for k, v in label_map.items()}

# Extrair as palavras e comparar com as predições
# Extrair as palavras e comparar com as predições
error_data = []
for idx, (true, pred) in enumerate(zip(true_labels, pred_labels)):
    for word_idx, (true_label, pred_label) in enumerate(zip(true, pred)):
        # Ignorar tokens com rótulo -100 (tokens de padding ou ignore)
        if true_label == -100:
            continue
        
        # Obter as labels correspondentes aos índices
        true_class = index_to_label.get(true_label, "Desconhecido")
        pred_class = index_to_label.get(pred_label, "Desconhecido")

        # Obter a palavra real a partir do ID de token
        word_id = test_set[idx]['input_ids'][word_idx]  # ID do token
        word = tokenizer.decode([word_id], skip_special_tokens=True)  # Decodifica para a palavra

        # Se a predição for errada, armazenar a palavra e as classificações
        if true_class != pred_class:
            error_data.append({
                'word': word,  # Agora 'word' é a palavra real
                'true_class': true_class,
                'pred_class': pred_class
            })

# Criar o DataFrame de erros
error_df = pd.DataFrame(error_data)

# Exibir o DataFrame com os erros cometidos
print(error_df)

# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/harem_second_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


         word true_class    pred_class
0    Gabinete    B-LOCAL             O
1           A    I-LOCAL             O
2         ##2    I-LOCAL             O
3           -    I-LOCAL             O
4          31    I-LOCAL             O
..        ...        ...           ...
448     ##ico   B-PESSOA  B-ABSTRACCAO
449         V   I-PESSOA      B-PESSOA
450         .   I-PESSOA      B-PESSOA
451         T   I-PESSOA      B-PESSOA
452         .   I-PESSOA      B-PESSOA

[453 rows x 3 columns]


In [8]:
# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/harem_second_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")


Arquivo salvo em: /home/juliaribeiro/harem_first_error_data.txt
